In [25]:
import os
from dotenv import load_dotenv
import google.generativeai
import gradio as gr
from IPython.display import Markdown, display
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi

In [26]:
load_dotenv()
YOUTUBE_API_KEY = os.getenv('YOUTUBE_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
google.generativeai.configure()

In [27]:
API_KEY = YOUTUBE_API_KEY 
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_video_id(url):
    """
    Extracts the video ID from a YouTube URL.
    """
    if "watch?v=" in url:
        return url.split("watch?v=")[-1].split("&")[0]
    elif "youtu.be/" in url:
        return url.split("youtu.be/")[-1].split("?")[0]
    return None

def get_subtitles(video_url, language='en'):
    """
    Fetches subtitles for a given YouTube video.

    Parameters:
    - video_url (str): The full URL of the YouTube video.
    - language (str): The language code for subtitles (default: 'en').

    Returns:
    - str: The extracted subtitles as a string.
    """
    video_id = get_video_id(video_url)
    if not video_id:
        return "Invalid YouTube URL"

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        subtitles = "\n".join([entry['text'] for entry in transcript])
        return subtitles
    except Exception as e:
        return f"Error: {str(e)}"

In [28]:
def summarizer(url):
    
    system_message = """
    Role: You are an AI assistant designed to analyze video subtitles and generate structured, high-quality explanations based on the content. Your goal is to provide a comprehensive yet concise summary, ensuring clarity and engagement.  
    
    ### **Response Structure:**  
    
    1. **Title:(h1)** `{Video Name} Explained`  
    2. **Overview of the Video(h2):** A brief discussion of the video's content, summarizing the key topics and information presented.  
    3. **Key Concepts(h2):** A breakdown of the fundamental ideas covered in the video.  
    4. **Step-by-Step Guide(h2)** (Only for tutorial-based videos, otherwise don't include this section in response): A structured, easy-to-follow guide explaining the process or method demonstrated in the video. Go beyond the subtitles to enrich the learning experience by providing additional context and insights.    
    5. **Code Snippets(h2)**: Important Note: Only for coding-based videos, if not coding video or coding related video don't include this section in response : Sequential, well-formatted examples with explanations to aid understanding.Note: If code snippets are provided in the Step-by-Step Guide then provide the whole code of those snippets in this section. Also explain the code in step-by-step manner using comments within the snippets.
    6. **Detailed Explanation:(h2)** A deep dive into the information presented, avoiding references to the video itself while maintaining a naturally described explanation.  
    7. **Conclusion:(h2)** A concise summary of the knowledge or insights gained from the video.  
    
    ### **Guidelines:**  
    - Be concise but thorough, ensuring clear and engaging explanations.  
    - Avoid unnecessary fluff—focus on actionable insights and direct explanations.  
    - If examples are provided in the video, include and clarify them. If unclear, create your own relevant examples.  
    - Go beyond the subtitles to enrich the learning experience by providing additional context and insights.  
    - Respond in Markdown
    """
    
    user_prompt = ("These are the subtitles of a video:\n" + 
                   get_subtitles(url))

    gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
    )
    response = gemini.generate_content(user_prompt)
    return response.text

In [29]:
with gr.Blocks(theme=gr.themes.Soft(), css="""
    .gradio-container { 
        max-width: 700px; 
        margin: auto; 
    }
    .response-text { 
        font-size: 1.2em; 
        font-weight: 500; 
    }
""") as view:
    
    gr.Markdown("# 🎥 YouTube Video Summarizer")
    gr.Markdown("Paste the URL of a YouTube video, and the AI will generate a structured summary.")
    
    with gr.Row():
        video_input = gr.Textbox(label="📌 Paste YouTube Video URL:", 
                                 placeholder="https://www.youtube.com/watch?v=example", interactive=True)
    
    submit_btn = gr.Button("Summarize 🔍")
    
    response_output = gr.Markdown(label="📄 Response:", elem_classes=["response-text"])

    submit_btn.click(fn=summarizer, inputs=[video_input], outputs=[response_output])

view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
